In [64]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from scipy.special import logit

In [80]:
# Load classifier training data
X_train = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_wnb_HL/embeddings.npy')
y_train_full = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_wnb_HL/labels.npy')
context_train = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_wnb_HL/correlations.npy')

# Load classifier + NF test data (signals)
X_test = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_signals_HL/embeddings.npy')
y_test_full = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_signals_HL/labels.npy')
context_test = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_signals_HL/correlations.npy')

# Load second test set (background)
X_bkg = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_HL/embeddings.npy')
y_bkg_full = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_HL/labels.npy')
context_bkg = np.load('/home/katya.govorkova/gwak2/gwak/output/ResNet_HL/correlations.npy')

# Print shapes for sanity check
print("X_train:", X_train.shape)
print("y_train:", y_train_full.shape)
print("context_train:", context_train.shape)

print("X_test (signals):", X_test.shape)
print("y_test (signals):", y_test_full.shape)
print("context_test (signals):", context_test.shape)

print("X_bkg:", X_bkg.shape)
print("y_bkg:", y_bkg_full.shape)
print("context_bkg:", context_bkg.shape)

X_train: (199936, 16)
y_train: (199936,)
context_train: (199936, 1)
X_test (signals): (199936, 16)
y_test (signals): (199936,)
context_test (signals): (199936, 1)
X_bkg: (99968, 16)
y_bkg: (99968,)
context_bkg: (99968, 1)


In [81]:
# Define binary labels for training set
train_signal_label = np.min(y_train_full)
y_train = (y_train_full == train_signal_label).astype(int)

# Save full test set before filtering
X_test_full = X_test.copy()
y_test_full_labels = y_test_full.copy()
context_test_full = context_test.copy()

# Define binary labels and filter for test set
test_signal_label = 8
test_background_labels = [10, 11]
test_mask = np.isin(y_test_full, [test_signal_label] + test_background_labels)

X_test = X_test[test_mask]
y_test_full = y_test_full[test_mask]
y_test = (y_test_full == test_signal_label).astype(int)
context_test = context_test[test_mask]  # <-- Apply same mask to context

print(f"[Train] Signals: {np.sum(y_train==1)}, Background: {np.sum(y_train==0)}")
print(f"[Test]  Signals: {np.sum(y_test==1)}, Background: {np.sum(y_test==0)}")

[Train] Signals: 99968, Background: 99968
[Test]  Signals: 24992, Background: 24992


In [82]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

# Concatenate embedding and context as input features
X_train_combined = np.concatenate([X_train, context_train], axis=1)
X_test_combined = np.concatenate([X_test, context_test], axis=1)
X_bkg_combined = np.concatenate([X_bkg, context_bkg], axis=1)  # NEW

# Split training set into train and validation
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_combined, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Create TensorDatasets
train_ds = TensorDataset(torch.tensor(X_tr, dtype=torch.float32),
                         torch.tensor(y_tr, dtype=torch.float32))
val_ds = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                       torch.tensor(y_val, dtype=torch.float32))
test_ds = TensorDataset(torch.tensor(X_test_combined, dtype=torch.float32),
                        torch.tensor(y_test, dtype=torch.float32))
bkg_ds = TensorDataset(torch.tensor(X_bkg_combined, dtype=torch.float32),
                       torch.tensor(y_bkg_full, dtype=torch.float32))  # NEW

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128)
test_loader = DataLoader(test_ds, batch_size=128)
bkg_loader = DataLoader(bkg_ds, batch_size=128)  # NEW

# Also save test context tensor separately for NF usage
context_train_tensor = torch.tensor(context_train, dtype=torch.float32)
context_test_tensor = torch.tensor(context_test, dtype=torch.float32)
context_bkg_tensor = torch.tensor(context_bkg, dtype=torch.float32)  # NEW

In [83]:
import pandas as pd

def summarize_features(X, name):
    df = pd.DataFrame(X, columns=[f'feat_{i}' for i in range(X.shape[1])])
    summary = pd.DataFrame({
        'min': df.min(),
        'max': df.max(),
        'mean': df.mean()
    })
    print(f"\n{name} feature summary:")
    display(summary)

# Summarize training features
summarize_features(X_train_combined, "Training set")

# Summarize test signal features
summarize_features(X_test_combined, "Test set (signals)")

# Summarize test background features
summarize_features(X_bkg_combined, "Test set (background)")


Training set feature summary:


,min,max,mean
feat_0,-0.906202,1.768859,0.296277
feat_1,-1.828741,0.331600,-0.955492
feat_2,-0.270942,2.227935,1.232510
feat_3,-3.631402,-0.525541,-2.478959
feat_4,-0.792154,1.708964,0.566783
feat_5,-2.176033,0.663829,-1.135372
feat_6,-2.957600,0.499910,-0.885186
feat_7,-6.715720,-2.348159,-5.540005
feat_8,-0.803847,0.340612,-0.237461
feat_9,-2.548387,0.716190,-0.864265



Test set (signals) feature summary:


,min,max,mean
feat_0,-0.884004,1.712306,0.293583
feat_1,-1.702500,0.385448,-0.953610
feat_2,-0.277713,2.224108,1.231197
feat_3,-3.562857,-0.865574,-2.478424
feat_4,-0.727057,1.710533,0.566758
feat_5,-2.194729,0.666447,-1.134076
feat_6,-2.957579,0.447521,-0.887502
feat_7,-6.692633,-2.577528,-5.540372
feat_8,-0.830857,0.341462,-0.237391
feat_9,-2.464218,0.676337,-0.861810



Test set (background) feature summary:


,min,max,mean
feat_0,-0.923425,1.544518,0.075557
feat_1,-1.602362,0.356237,-0.830534
feat_2,-0.286450,2.032506,1.101465
feat_3,-3.369917,-0.613869,-2.414099
feat_4,-0.765827,1.727490,0.595974
feat_5,-1.965027,0.666447,-1.001327
feat_6,-2.957600,0.246266,-1.106921
feat_7,-6.462726,-2.335717,-5.530254
feat_8,-0.782748,0.341462,-0.226501
feat_9,-2.187151,0.736296,-0.677595


In [69]:
# # Define better MLP classifier
# class BetterMLPClassifier(nn.Module):
#     def __init__(self, input_dim):
#         super().__init__()
#         self.model = nn.Sequential(
#             nn.Linear(input_dim, 256),
#             nn.BatchNorm1d(256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.BatchNorm1d(128),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(128, 64),
#             nn.BatchNorm1d(64),
#             nn.ReLU(),
#             nn.Linear(64, 1)
#         )

#     def forward(self, x):
#         return self.model(x).squeeze()


# # Margin-based binary contrastive loss
# class MarginBinaryLoss(nn.Module):
#     def __init__(self, margin=0.2):
#         super().__init__()
#         self.margin = margin
#         self.bce = nn.BCEWithLogitsLoss()

#     def forward(self, logits, targets):
#         loss = self.bce(logits, targets)

#         signal_mask = targets == 1
#         background_mask = targets == 0

#         if signal_mask.sum() > 0 and background_mask.sum() > 0:
#             signal_logits = logits[signal_mask]
#             background_logits = logits[background_mask]

#             hardest_background = background_logits.max()
#             margin_loss = torch.relu(self.margin - (signal_logits - hardest_background)).mean()

#             loss += margin_loss
#         return loss

In [70]:
# # Setup device, model, optimizer, loss, scaler, and scheduler
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = BetterMLPClassifier(input_dim=X_train_combined.shape[1]).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# criterion = MarginBinaryLoss(margin=0.2)
# scaler = torch.cuda.amp.GradScaler(enabled=(device == 'cuda'))
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# # Training loop with validation loss
# for epoch in range(1, 21):
#     model.train()
#     running_loss = 0.0

#     for xb, yb in train_loader:
#         xb, yb = xb.to(device), yb.to(device)

#         optimizer.zero_grad()
#         with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
#             logits = model(xb)
#             loss = criterion(logits, yb)

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#         running_loss += loss.item()

#     avg_loss = running_loss / len(train_loader)

#     # Validation loss
#     model.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for xb, yb in val_loader:
#             xb, yb = xb.to(device), yb.to(device)
#             with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
#                 logits = model(xb)
#                 loss = criterion(logits, yb)
#             val_loss += loss.item()

#     avg_val_loss = val_loss / len(val_loader)
#     scheduler.step(avg_val_loss)

#     print(f"Epoch {epoch:02d}, Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define better MLP classifier
class BetterMLPClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x).squeeze()

# Focal loss definition
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        targets = targets.float()
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        probas = torch.sigmoid(logits)
        p_t = probas * targets + (1 - probas) * (1 - targets)
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        focal_loss = alpha_t * (1 - p_t) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Combined classifier + NF loss
class ClassifierNFJointLoss(nn.Module):
    def __init__(self, margin=0.2, alpha=1.0, beta=1.0, focal_alpha=0.25, focal_gamma=2.0):
        super().__init__()
        self.margin = margin
        self.alpha = alpha  # weight for margin loss
        self.beta = beta    # weight for NF penalty
        self.focal = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)

    def forward(self, logits, targets, nf_log_likelihood):
        targets = targets.float()
        loss = self.focal(logits, targets)

        signal_mask = targets == 1
        background_mask = targets == 0

        if signal_mask.sum() > 0 and background_mask.sum() > 0:
            signal_logits = logits[signal_mask]
            background_logits = logits[background_mask]
            hardest_background = background_logits.max()
            margin_loss = torch.relu(self.margin - (signal_logits - hardest_background)).mean()
            loss += self.alpha * margin_loss

        if nf_log_likelihood is not None:
            nf_bkg = nf_log_likelihood[background_mask]
            nf_signal = nf_log_likelihood[signal_mask]
            nf_loss = (nf_signal.mean() - nf_bkg.mean())
            loss -= self.beta * nf_loss

        return loss

# Setup device, model, optimizer, loss, scaler, and scheduler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BetterMLPClassifier(input_dim=X_train_combined.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Use joint classifier + NF loss with focal loss
criterion = ClassifierNFJointLoss(margin=0.2, alpha=1.0, beta=0.01)

scaler = torch.cuda.amp.GradScaler(enabled=(device == 'cuda'))
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Training loop with validation loss
for epoch in range(1, 21):
    model.train()
    running_loss = 0.0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        x_emb = xb[:, :-1]  # embedding
        x_ctx = xb[:, -1:]  # context

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
            logits = model(xb)

            with torch.no_grad():
                nf_ll = -1 * nf_model(x_emb, x_ctx)

            loss = criterion(logits, yb, nf_ll)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # Validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)

            x_emb = xb[:, :-1]
            x_ctx = xb[:, -1:]

            with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
                logits = model(xb)
                nf_ll = -1 * nf_model(x_emb, x_ctx)
                loss = criterion(logits, yb, nf_ll)

            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    scheduler.step(avg_val_loss)
    print(f"Epoch {epoch:02d}, Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

/local/katya.govorkova/ipykernel_95553/2762259159.py:89: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == 'cuda'))
/local/katya.govorkova/ipykernel_95553/2762259159.py:104: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
/local/katya.govorkova/ipykernel_95553/2762259159.py:131: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == 'cuda')):


Epoch 01, Train Loss: 0.2504, Val Loss: 0.1927
Epoch 02, Train Loss: 0.2389, Val Loss: 0.2906
Epoch 03, Train Loss: 0.2625, Val Loss: 0.1991
Epoch 04, Train Loss: 0.2800, Val Loss: 0.2123
Epoch 05, Train Loss: 0.2853, Val Loss: 0.2381
Epoch 06, Train Loss: 0.2882, Val Loss: 0.2535
Epoch 07, Train Loss: 0.2864, Val Loss: 0.2820
Epoch 08, Train Loss: 0.2854, Val Loss: 0.2726
Epoch 09, Train Loss: 0.2873, Val Loss: 0.2420
Epoch 10, Train Loss: 0.2899, Val Loss: 0.2400
Epoch 11, Train Loss: 0.2772, Val Loss: 0.2816
Epoch 12, Train Loss: 0.2750, Val Loss: 0.2550
Epoch 13, Train Loss: 0.2730, Val Loss: 0.2695
Epoch 14, Train Loss: 0.2840, Val Loss: 0.2842
Epoch 15, Train Loss: 0.2711, Val Loss: 0.2647
Epoch 16, Train Loss: 0.2706, Val Loss: 0.2574
Epoch 17, Train Loss: 0.2652, Val Loss: 0.2728
Epoch 18, Train Loss: 0.2882, Val Loss: 0.2569
Epoch 19, Train Loss: 0.2722, Val Loss: 0.2464
Epoch 20, Train Loss: 0.2744, Val Loss: 0.2374


In [ ]:
# Evaluate classifier on test set
model.eval()
y_test_logits, y_test_probs, y_test_true = [], [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb)
        probs = torch.sigmoid(logits)

        y_test_logits.append(logits.cpu().numpy())
        y_test_probs.append(probs.cpu().numpy())
        y_test_true.append(yb.numpy())

# Concatenate results
y_test_logits = np.concatenate(y_test_logits)
y_test_probs = np.concatenate(y_test_probs)
y_test_true = np.concatenate(y_test_true)

# Metrics
roc_auc = roc_auc_score(y_test_true, y_test_probs)
acc = accuracy_score(y_test_true, y_test_probs > 0.5)

print(f"[TEST] AUC: {roc_auc:.3f}, Accuracy: {acc:.3f}")

In [ ]:
model.eval()
bkg_probs = []

with torch.no_grad():
    for xb, _ in bkg_loader:
        xb = xb.to(device)
        probs = torch.sigmoid(model(xb))
        bkg_probs.append(probs.cpu().numpy())

bkg_probs = np.concatenate(bkg_probs)

plt.hist(bkg_probs, bins=50, color='orange', alpha=0.7)
plt.title("Classifier Scores on Background-Only Set")
plt.xlabel("Score")
plt.ylabel("Count")
plt.grid()
plt.show()

print(f"Background scores > 0.5: {(bkg_probs > 0.5).sum()} / {len(bkg_probs)}")

In [ ]:
# Load trained NF
nf_model = torch.jit.load("/home/katya.govorkova/gwak2/gwak/output/ResNet_NF_from_file_conditioning_HL/model_JIT.pt").eval().to(device)

# Evaluate NF log-probabilities on test set
nf_scores = []

with torch.no_grad():
    for i in range(0, len(X_test), 512):
        xb = torch.tensor(X_test[i:i+512], dtype=torch.float32).to(device)           # standardized embeddings
        ctx = context_test_tensor[i:i+512].to(device)                                # corresponding context
        log_probs = nf_model(xb, context=ctx) * (-1)                               # assumes .log_prob(x, context=...)
        nf_scores.append(log_probs.cpu().numpy())

nf_scores = np.concatenate(nf_scores)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# Signal of interest
signal_label = 8
background_labels = [10, 11]

# Mask for selected signal + background
mask = np.isin(y_test_full, [signal_label] + background_labels)
y_bin_true = (y_test_full[mask] == signal_label).astype(int)

# Classifier-based scores
clf_scores = y_test_probs[mask]

# NF anomaly scores (lower = more anomalous, so invert)
nf_anomaly_scores = nf_scores[mask]

# Plot
plt.figure(figsize=(8, 6))

# Classifier ROC
fpr_clf, tpr_clf, _ = roc_curve(y_bin_true, clf_scores)
auc_clf = roc_auc_score(y_bin_true, clf_scores)
plt.plot(fpr_clf, tpr_clf, label=f"Classifier (AUC={auc_clf:.2f})")

# NF ROC
fpr_nf, tpr_nf, _ = roc_curve(y_bin_true, nf_anomaly_scores)
auc_nf = roc_auc_score(y_bin_true, nf_anomaly_scores)
plt.plot(fpr_nf, tpr_nf, label=f"NF (AUC={auc_nf:.2f})")

# Plot formatting
plt.plot([0, 1], [0, 1], '--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Signal 8 (WNB) vs Background (10,11)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Log-scale histogram of predicted probabilities
plt.figure(figsize=(8, 5))
plt.hist(y_test_probs[y_test_true == 1], bins=100, alpha=0.6, label='Signal', density=True)
plt.hist(y_test_probs[y_test_true == 0], bins=100, alpha=0.6, label='Background', density=True)
plt.xscale('log')
plt.xlabel('Prediction Score (log scale)')
plt.ylabel('Density')
plt.title('Classifier Score Tail (log scale)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=False)

# --- Left: Classifier ---
axs[0].hist(y_test_probs[y_test_true == 1], bins=100, alpha=0.6, label=f'Signal, {max(y_test_probs[y_test_true == 1]):.2f}', density=True)
axs[0].hist(y_test_probs[y_test_true == 0], bins=100, alpha=0.6, label=f'Background, {max(y_test_probs[y_test_true == 0]):.2f}', density=True)
axs[0].set_xlabel('Classifier Score (log scale)')
axs[0].set_ylabel('Density')
axs[0].set_title('Classifier Output')
axs[0].legend()
axs[0].grid(True)

# --- Right: NF (clipped) ---
axs[1].hist(nf_scores[y_test_true == 1], bins=100, alpha=0.6, label=f'Signal, {max(nf_scores[y_test_true == 1]):.2f}', density=True)
axs[1].hist(nf_scores[y_test_true == 0], bins=100, alpha=0.6, label=f'Background, {max(nf_scores[y_test_true == 0]):.2f}', density=True)
axs[1].set_xlabel('−Log-Likelihood (log scale)')
axs[1].set_title('Normalizing Flow Output')
axs[1].legend()
axs[1].grid(True)

plt.suptitle("Signal vs Background Score Distributions (log-scale)")
plt.tight_layout()
plt.show()

In [ ]:
min(y_test_probs[y_test_true == 1]), max(y_test_probs[y_test_true == 1])

In [ ]:
min(y_test_probs[y_test_true == 0]), max(y_test_probs[y_test_true == 0])

In [ ]:
from gwak.train.plotting import make_corner
fig = make_corner(X_train, (y_train).astype(int), return_fig=True, label_names=['WNB', 'Background'])